In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare, kstest, normaltest, shapiro

In [28]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
df = pd.read_csv('source/iris.txt', names=column_names)
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [29]:
class_mapping = {
    'Iris-setosa': 0,
    'Iris-versicolor': 1,
    'Iris-virginica': 2
}

df['class'] = df['class'].replace(class_mapping)
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [30]:
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]
class_2 = df[df['class'] == 2]

In [31]:
from scipy.stats import ks_2samp

columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

for column in columns:
    for class_pair in [(class_0, class_1), (class_0, class_2), (class_1, class_2)]:
        class_1_p, class_2_p = class_pair
        stat, p_value = ks_2samp(class_1_p[column], class_2_p[column])
        class_1_name = class_1_p['class'].unique()
        class_2_name = class_2_p['class'].unique()
        print(f"Тест Колмогорова-Смирнова для {column} между классом {class_1_name} и классом {class_2_name}:")

        if p_value < 0.05:
            print(f"Распределения {column} различаются между классом {class_1_name} и классом {class_2_name}, p-value={p_value}")
        else:
            print(f"Распределения {column} схожи между классом {class_1_name} и классом {class_2_name}, p-value={p_value}")

Тест Колмогорова-Смирнова для sepal_length между классом [0] и классом [1]:
Распределения sepal_length различаются между классом [0] и классом [1], p-value=2.807570962237254e-15
Тест Колмогорова-Смирнова для sepal_length между классом [0] и классом [2]:
Распределения sepal_length различаются между классом [0] и классом [2], p-value=7.773164323782225e-23
Тест Колмогорова-Смирнова для sepal_length между классом [1] и классом [2]:
Распределения sepal_length различаются между классом [1] и классом [2], p-value=3.800827929128319e-05
Тест Колмогорова-Смирнова для sepal_width между классом [0] и классом [1]:
Распределения sepal_width различаются между классом [0] и классом [1], p-value=2.6679407140599687e-11
Тест Колмогорова-Смирнова для sepal_width между классом [0] и классом [2]:
Распределения sepal_width различаются между классом [0] и классом [2], p-value=4.8075337049514946e-06
Тест Колмогорова-Смирнова для sepal_width между классом [1] и классом [2]:
Распределения sepal_width схожи между

Видим, что функции распределения равны только у sepal_width классов 1 и 2

Проверим теперь равенство дисперсий

In [32]:
from scipy.stats import f

columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

results = []

for column in columns:
    for class_pair in [(class_0, class_1), (class_0, class_2), (class_1, class_2)]:
        class_1_p, class_2_p = class_pair
        class_1_name = class_1_p['class'].unique()
        class_2_name = class_2_p['class'].unique()
        var1 = class_1_p[column].var(ddof=1)
        var2 = class_2_p[column].var(ddof=1)
        F = var1 / var2

        alpha = 0.05

        n = len(class_1_p) - 1
        m = len(class_2_p) - 1

        critical_value = f.ppf(1 - alpha / 2, n, m)

        results.append((column, class_1_name, class_2_name, F, critical_value))

for column, class1, class2, F, critical_value in results:
    print(f"Тест Фишера для дисперсии {column} между классом {class1} и классом {class2}:")
    if F < critical_value:
        print(f"Равенство дисперсий не отвергается")
    else:
        print(f"Равенство дисперсий отвергается")

Тест Фишера для дисперсии sepal_length между классом [0] и классом [1]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии sepal_length между классом [0] и классом [2]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии sepal_length между классом [1] и классом [2]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии sepal_width между классом [0] и классом [1]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии sepal_width между классом [0] и классом [2]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии sepal_width между классом [1] и классом [2]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии petal_length между классом [0] и классом [1]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии petal_length между классом [0] и классом [2]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии petal_length между классом [1] и классом [2]:
Равенство дисперсий не отвергается
Тест Фишера для дисперсии petal_width ме

Дисперсии попарно равны, проверим их общее равенство:

In [33]:
from scipy.stats import bartlett

columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

results = []

for column in columns:
    samples = [class_0[column], class_1[column], class_2[column]]
    statistic, p_value = bartlett(*samples)
    results.append((column, statistic, p_value))

for column, statistic, p_value in results:
    print(f"Тест Бартлетта для дисперсии {column} между всеми классами:")
    print(f"Статистика: {statistic}")
    print(f"p-значение: {p_value}")

    if p_value < 0.05:
        print(f"Равенство дисперсий не отвергается для столбца {column}")
    else:
        print(f"Равенство дисперсий отвергается для столбца {column}")


Тест Бартлетта для дисперсии sepal_length между всеми классами:
Статистика: 16.005701874401502
p-значение: 0.0003345076070163035
Равенство дисперсий не отвергается для столбца sepal_length
Тест Бартлетта для дисперсии sepal_width между всеми классами:
Статистика: 2.2158125491551637
p-значение: 0.3302496898960959
Равенство дисперсий отвергается для столбца sepal_width
Тест Бартлетта для дисперсии petal_length между всеми классами:
Статистика: 55.49409813024258
p-значение: 8.904503355816222e-13
Равенство дисперсий не отвергается для столбца petal_length
Тест Бартлетта для дисперсии petal_width между всеми классами:
Статистика: 37.99553767573184
p-значение: 5.615311140768039e-09
Равенство дисперсий не отвергается для столбца petal_width


Дисперсии не равны только в столбце sepal_width

In [35]:
from scipy.stats import ttest_ind

columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
mean_results = []
for column in columns:
    for class_pair in [(class_0, class_1), (class_0, class_2), (class_1, class_2)]:
        class_1_p, class_2_p = class_pair
        class_1_name = class_1_p['class'].unique()
        class_2_name = class_2_p['class'].unique()
        t_stat, t_p_value = ttest_ind(class_1_p[column], class_2_p[column])

        mean_results.append((column, class_1_name, class_2_name, t_stat, t_p_value))

for column, class1, class2, t_stat, t_p_value in mean_results:
    print(f"Тест Стьюдента для средних {column} между классом {class1} и классом {class2}:")
    if t_p_value < 0.05:
        print("Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value =", t_p_value)
    else:
        print("Средние значения выборок схожи (не отвергаем нулевую гипотезу), p-value =", t_p_value)


Тест Стьюдента для средних sepal_length между классом [0] и классом [1]:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 8.985235037487079e-18
Тест Стьюдента для средних sepal_length между классом [0] и классом [2]:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 6.892546060674059e-28
Тест Стьюдента для средних sepal_length между классом [1] и классом [2]:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 1.7248563024547942e-07
Тест Стьюдента для средних sepal_width между классом [0] и классом [1]:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 4.362239016010214e-15
Тест Стьюдента для средних sepal_width между классом [0] и классом [2]:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 8.916634067006441e-09
Тест Стьюдента для средних sepal_width между классом [1] и классом [2]:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-

Поскольку средние не равны попарно, нет смысла проверять равенство в совокупности.